In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, window, max
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

spark = (
    SparkSession.builder
    .appName("Streaming from Kafka with sliding window") 
    .config("spark.streaming.stopGracefullyOnShutdown", True)
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
        "org.apache.spark:spark-streaming-kafka-0-10-assembly_2.12:3.3.0"
    )
    .master("local[*]")
    .getOrCreate()
)
spark

In [2]:
# docker exec -it kafka kafka-console-producer --bootstrap-server localhost:9092 --topic trades3

In [ ]:
schema = StructType([
        StructField("CreatedTime", StringType()),
        StructField("Reading", DoubleType())
])

kafka_source_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "trades3")
    .option("startingOffsets", "earliest")
    .load()
)

schema = StructType([
        StructField("CreatedTime", StringType()),
        StructField("Reading", DoubleType())
])


from pyspark.sql.functions import col, expr, from_json

# Step 1: ubah value (binary) menjadi string dulu
df_str = kafka_source_df.withColumn("value_str", col("value").cast("string"))

# Step 2: cari posisi ':' pertama
with_pos = df_str.withColumn("pos", expr("instr(value_str, ':')"))

# Step 3: pisahkan prefix SensorID dan JSON string
split_df = with_pos.select(
    expr("substring(value_str, 1, pos - 1)").alias("SensorID"),
    expr("substring(value_str, pos + 1, length(value_str))").alias("json_str")
)

# Step 4: parse JSON
value_df = split_df.select(
    "SensorID",
    from_json(col("json_str"), schema).alias("value")
)

# value_df.show(truncate=False)

sensor_df = value_df.select("SensorID", "value.*") \
        .withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss"))

agg_df = sensor_df \
        .withWatermark("CreatedTime", "30 minute") \
        .groupBy(col("SensorID"),
                 window(col("CreatedTime"), "15 minute", "5 minute")) \
        .agg(max("Reading").alias("MaxReading"))

output_df = agg_df.select("SensorID", "window.start", "window.end", "MaxReading")



sensor_df = value_df.select("SensorID", "value.*") \
        .withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss"))

agg_df = sensor_df \
        .withWatermark("CreatedTime", "30 minute") \
        .groupBy(col("SensorID"),
                 window(col("CreatedTime"), "15 minute", "5 minute")) \
        .agg(max("Reading").alias("MaxReading"))

output_df = agg_df.select("SensorID", "window.start", "window.end", "MaxReading")

# output_df.show()

window_query = output_df.writeStream \
        .format("console") \
        .outputMode("update") \
        .option("checkpointLocation", "chk-point-dir") \
        .trigger(processingTime="1 minute") \
        .start() \
        .awaitTermination()


# window_query

In [4]:
# window_query.stop()

In [14]:
schema = StructType([
        StructField("CreatedTime", StringType()),
        StructField("Reading", DoubleType())
])

kafka_source_df = (
    spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "trades3")
    .option("startingOffsets", "earliest")
    .load()
)

from pyspark.sql.functions import col, expr, from_json

# Step 1: ubah value (binary) menjadi string dulu
df_str = kafka_source_df.withColumn("value_str", col("value").cast("string"))

# Step 2: cari posisi ':' pertama
with_pos = df_str.withColumn("pos", expr("instr(value_str, ':')"))

# Step 3: pisahkan prefix SensorID dan JSON string
split_df = with_pos.select(
    expr("substring(value_str, 1, pos - 1)").alias("SensorID"),
    expr("substring(value_str, pos + 1, length(value_str))").alias("json_str")
)

# Step 4: parse JSON
value_df = split_df.select(
    "SensorID",
    from_json(col("json_str"), schema).alias("value")
)

# value_df.show(truncate=False)

sensor_df = value_df.select("SensorID", "value.*") \
        .withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss"))

agg_df = sensor_df \
        .withWatermark("CreatedTime", "30 minute") \
        .groupBy(col("SensorID"),
                 window(col("CreatedTime"), "15 minute", "5 minute")) \
        .agg(max("Reading").alias("MaxReading"))

output_df = agg_df.select("SensorID", "window.start", "window.end", "MaxReading")



sensor_df = value_df.select("SensorID", "value.*") \
        .withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss"))

agg_df = sensor_df \
        .withWatermark("CreatedTime", "30 minute") \
        .groupBy(col("SensorID"),
                 window(col("CreatedTime"), "15 minute", "5 minute")) \
        .agg(max("Reading").alias("MaxReading"))

output_df = agg_df.select("SensorID", "window.start", "window.end", "MaxReading")

output_df.show()

+--------+-------------------+-------------------+----------+
|SensorID|              start|                end|MaxReading|
+--------+-------------------+-------------------+----------+
|   SET41|2019-02-05 10:10:00|2019-02-05 10:25:00|      37.7|
|   SET41|2019-02-05 10:25:00|2019-02-05 10:40:00|      37.2|
|   SET41|2019-02-05 09:40:00|2019-02-05 09:55:00|      36.2|
|   SET41|2019-02-05 09:45:00|2019-02-05 10:00:00|      36.5|
|   SET41|2019-02-05 10:00:00|2019-02-05 10:15:00|      36.8|
|   SET41|2019-02-05 10:20:00|2019-02-05 10:35:00|      37.7|
|   SET41|2019-02-05 10:15:00|2019-02-05 10:30:00|      37.7|
|   SET41|2019-02-05 10:05:00|2019-02-05 10:20:00|      36.5|
|   SET41|2019-02-05 09:50:00|2019-02-05 10:05:00|      36.8|
|   SET41|2019-02-05 09:55:00|2019-02-05 10:10:00|      36.8|
+--------+-------------------+-------------------+----------+



In [12]:
from pyspark.sql.functions import col, expr, from_json

# Step 1: ubah value (binary) menjadi string dulu
df_str = kafka_source_df.withColumn("value_str", col("value").cast("string"))

# Step 2: cari posisi ':' pertama
with_pos = df_str.withColumn("pos", expr("instr(value_str, ':')"))

# Step 3: pisahkan prefix SensorID dan JSON string
split_df = with_pos.select(
    expr("substring(value_str, 1, pos - 1)").alias("SensorID"),
    expr("substring(value_str, pos + 1, length(value_str))").alias("json_str")
)

# Step 4: parse JSON
value_df = split_df.select(
    "SensorID",
    from_json(col("json_str"), schema).alias("value")
)

# value_df.show(truncate=False)

sensor_df = value_df.select("SensorID", "value.*") \
        .withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss"))

agg_df = sensor_df \
        .withWatermark("CreatedTime", "30 minute") \
        .groupBy(col("SensorID"),
                 window(col("CreatedTime"), "15 minute", "5 minute")) \
        .agg(max("Reading").alias("MaxReading"))

output_df = agg_df.select("SensorID", "window.start", "window.end", "MaxReading")

output_df.show()

+--------+-------------------+-------------------+----------+
|SensorID|              start|                end|MaxReading|
+--------+-------------------+-------------------+----------+
|   SET41|2019-02-05 10:10:00|2019-02-05 10:25:00|      37.7|
|   SET41|2019-02-05 10:25:00|2019-02-05 10:40:00|      37.2|
|   SET41|2019-02-05 09:40:00|2019-02-05 09:55:00|      36.2|
|   SET41|2019-02-05 09:45:00|2019-02-05 10:00:00|      36.5|
|   SET41|2019-02-05 10:00:00|2019-02-05 10:15:00|      36.8|
|   SET41|2019-02-05 10:20:00|2019-02-05 10:35:00|      37.7|
|   SET41|2019-02-05 10:15:00|2019-02-05 10:30:00|      37.7|
|   SET41|2019-02-05 10:05:00|2019-02-05 10:20:00|      36.5|
|   SET41|2019-02-05 09:50:00|2019-02-05 10:05:00|      36.8|
|   SET41|2019-02-05 09:55:00|2019-02-05 10:10:00|      36.8|
+--------+-------------------+-------------------+----------+

